In [17]:
import pandas as pd
import geopandas as gpd
from datetime import datetime, date
import requests
import json

s3_output_path = 'pathogens/cdc/nndss/output'

mpox_url="https://data.cdc.gov/resource/x9gk-5huc.geojson?$query=SELECT%0A%20%20%60states%60%2C%0A%20%20%60year%60%2C%0A%20%20%60week%60%2C%0A%20%20%60label%60%2C%0A%20%20%60m1%60%2C%0A%20%20%60m1_flag%60%2C%0A%20%20%60m2%60%2C%0A%20%20%60m2_flag%60%2C%0A%20%20%60m3%60%2C%0A%20%20%60m3_flag%60%2C%0A%20%20%60m4%60%2C%0A%20%20%60m4_flag%60%2C%0A%20%20%60location1%60%2C%0A%20%20%60location2%60%2C%0A%20%20%60sort_order%60%2C%0A%20%20%60geocode%60%0AWHERE%20caseless_one_of(%60label%60%2C%20%22Mpox%22)%0AORDER%20BY%20%60sort_order%60%20ASC%20NULL%20LAST"


In [5]:
mpox_df=gpd.read_file(mpox_url)
mpox_df

,m1_flag,m2,m4_flag,m2_flag,m4,label,year,m1,m3_flag,week,location2,location1,m3,sort_order,states,geometry
0,-,70,-,-,51,Mpox,2024,11,-,1,US RESIDENTS,None,11,20240104971,US RESIDENTS,None
1,-,5,-,-,None,Mpox,2024,None,-,1,NEW ENGLAND,None,None,20240104972,NEW ENGLAND,None
2,-,1,-,-,None,Mpox,2024,None,-,1,None,CONNECTICUT,None,20240104973,CONNECTICUT,POINT (-72.73831 41.5735)
3,-,0,-,-,None,Mpox,2024,None,-,1,None,MAINE,None,20240104974,MAINE,POINT (-69.20395 45.31525)
4,-,4,-,-,None,Mpox,2024,None,-,1,None,MASSACHUSETTS,None,20240104975,MASSACHUSETTS,POINT (-71.71818 42.35867)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-,5,-,-,2,Mpox,2024,None,-,15,None,INDIANA,5,20241504986,INDIANA,POINT (-86.28179 39.91981)
996,-,3,-,-,4,Mpox,2024,1,-,15,None,MICHIGAN,16,20241504987,MICHIGAN,POINT (-84.1967 44.1252)
997,-,7,-,-,5,Mpox,2024,None,-,15,None,OHIO,14,20241504988,OHIO,POINT (-82.71122 40.41306)
998,-,2,-,-,1,Mpox,2024,None,-,15,None,WISCONSIN,None,20241504989,WISCONSIN,POINT (-89.73919 44.64483)


In [10]:
mpox_df["lat"] = mpox_df.geometry.y
mpox_df["lon"] = mpox_df.geometry.x
mpox_df['date'] = mpox_df.apply(lambda row: date.fromisocalendar(int(row['year']),int(row['week']), 1), axis=1 )
mpox_df['date'] = pd.to_datetime(mpox_df['date'])
mpox_df['current_week'] = mpox_df['m1']
mpox_df['previous_52_weeks_max'] = mpox_df['m2']
mpox_df['current_YTD_cummulative'] = mpox_df['m3']
mpox_df['previous_YTD_cummulative'] = mpox_df['m4']
mpox_df

,m1_flag,m2,m4_flag,m2_flag,m4,label,year,m1,m3_flag,week,...,sort_order,states,geometry,lat,lon,date,current_week,previous_52_weeks_max,current_YTD_cummulative,previous_YTD_cummulative
0,-,70,-,-,51,Mpox,2024,11,-,1,...,20240104971,US RESIDENTS,None,NaN,NaN,2024-01-01,11,70,11,51
1,-,5,-,-,None,Mpox,2024,None,-,1,...,20240104972,NEW ENGLAND,None,NaN,NaN,2024-01-01,None,5,None,None
2,-,1,-,-,None,Mpox,2024,None,-,1,...,20240104973,CONNECTICUT,POINT (-72.73831 41.5735),41.573503,-72.738306,2024-01-01,None,1,None,None
3,-,0,-,-,None,Mpox,2024,None,-,1,...,20240104974,MAINE,POINT (-69.20395 45.31525),45.315250,-69.203947,2024-01-01,None,0,None,None
4,-,4,-,-,None,Mpox,2024,None,-,1,...,20240104975,MASSACHUSETTS,POINT (-71.71818 42.35867),42.358670,-71.718185,2024-01-01,None,4,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-,5,-,-,2,Mpox,2024,None,-,15,...,20241504986,INDIANA,POINT (-86.28179 39.91981),39.919807,-86.281793,2024-04-08,None,5,5,2
996,-,3,-,-,4,Mpox,2024,1,-,15,...,20241504987,MICHIGAN,POINT (-84.1967 44.1252),44.125200,-84.196697,2024-04-08,1,3,16,4
997,-,7,-,-,5,Mpox,2024,None,-,15,...,20241504988,OHIO,POINT (-82.71122 40.41306),40.413057,-82.711215,2024-04-08,None,7,14,5
998,-,2,-,-,1,Mpox,2024,None,-,15,...,20241504989,WISCONSIN,POINT (-89.73919 44.64483),44.644833,-89.739185,2024-04-08,None,2,None,1


In [11]:
mpox_clean_df = mpox_df.dropna(subset=["lat", "lon"])

mpox_clean_df

,m1_flag,m2,m4_flag,m2_flag,m4,label,year,m1,m3_flag,week,...,sort_order,states,geometry,lat,lon,date,current_week,previous_52_weeks_max,current_YTD_cummulative,previous_YTD_cummulative
2,-,1,-,-,None,Mpox,2024,None,-,1,...,20240104973,CONNECTICUT,POINT (-72.73831 41.5735),41.573503,-72.738306,2024-01-01,None,1,None,None
3,-,0,-,-,None,Mpox,2024,None,-,1,...,20240104974,MAINE,POINT (-69.20395 45.31525),45.315250,-69.203947,2024-01-01,None,0,None,None
4,-,4,-,-,None,Mpox,2024,None,-,1,...,20240104975,MASSACHUSETTS,POINT (-71.71818 42.35867),42.358670,-71.718185,2024-01-01,None,4,None,None
5,-,0,-,-,None,Mpox,2024,None,-,1,...,20240104976,NEW HAMPSHIRE,POINT (-71.5776 43.68553),43.685525,-71.577597,2024-01-01,None,0,None,None
6,-,1,-,-,None,Mpox,2024,None,-,1,...,20240104977,RHODE ISLAND,POINT (-71.45432 41.48871),41.488713,-71.454319,2024-01-01,None,1,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,-,14,-,-,12,Mpox,2024,None,-,15,...,20241504985,ILLINOIS,POINT (-89.15865 40.11405),40.114047,-89.158651,2024-04-08,None,14,64,12
995,-,5,-,-,2,Mpox,2024,None,-,15,...,20241504986,INDIANA,POINT (-86.28179 39.91981),39.919807,-86.281793,2024-04-08,None,5,5,2
996,-,3,-,-,4,Mpox,2024,1,-,15,...,20241504987,MICHIGAN,POINT (-84.1967 44.1252),44.125200,-84.196697,2024-04-08,1,3,16,4
997,-,7,-,-,5,Mpox,2024,None,-,15,...,20241504988,OHIO,POINT (-82.71122 40.41306),40.413057,-82.711215,2024-04-08,None,7,14,5


In [29]:
url="https://data.cdc.gov/resource/x9gk-5huc.geojson?$select=count(year)&year=2022"
response=requests.get(url)
count=response.json()
count 

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': None,
   'properties': {'count_year': '400400'}}],
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}}

In [30]:
url="https://data.cdc.gov/resource/x9gk-5huc.geojson?year=2022"
response=requests.get(url)
data=response.json()
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': None,
   'properties': {'m1_flag': '-',
    'm2': '0',
    'm4_flag': '-',
    'm2_flag': '-',
    'm4': None,
    'label': 'Anthrax',
    'year': '2022',
    'm1': None,
    'm3_flag': '-',
    'week': '1',
    'location2': 'US RESIDENTS',
    'location1': None,
    'm3': None,
    'sort_order': '20220100001',
    'states': 'US RESIDENTS'}},
  {'type': 'Feature',
   'geometry': None,
   'properties': {'m1_flag': '-',
    'm2': '0',
    'm4_flag': '-',
    'm2_flag': '-',
    'm4': None,
    'label': 'Anthrax',
    'year': '2022',
    'm1': None,
    'm3_flag': '-',
    'week': '1',
    'location2': 'NEW ENGLAND',
    'location1': None,
    'm3': None,
    'sort_order': '20220100002',
    'states': 'NEW ENGLAND'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-72.738288, 41.575155]},
   'properties': {'m1_flag': '-',
    'm2': '0',
    'm4_flag': '-',
    'm2_flag': '-',
    'm4': N